# 0. Install Dependencies

In [13]:
!pip install torch
!pip install torchvision


[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# 1. Import required libraries

In [14]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib as plt
from sklearn.metrics import confusion_matrix, accuracy_score


# 2. Load the dataset

In [16]:
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

test_val_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

train_dataset = ImageFolder(root='archive/train', transform=train_transform)
test_dataset = ImageFolder(root='archive/test', transform=test_val_transform)

batch_size = 4
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


# 3. Define the CNN Model


In [17]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = CNN()


# 4. Define Loss Function and Optimizer


In [18]:
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# 5. Define Training Function

In [19]:
def train_model(epoch_num):
    epoch_train_loss = []
    epoch_test_loss = []
    for epoch in range(epoch_num):
        train_loss = []
        model.train()
        for batch_idx, (image, label) in enumerate(train_loader):
            image = image.to("cuda")
            label = label.to("cuda")
            optimizer.zero_grad()
            # feed forword
            output = model(image)
            # calculate loss
            loss = loss_function(output, label)
            # calculate direviate
            loss.backward()
            # Performs a single optimization step (parameter update).
            optimizer.step()
            train_loss.append(loss.item())
        print(f'Epoch [{epoch + 1}/{epoch_num}], Train Loss: {np.mean(train_loss):.4f}')
    
        epoch_train_loss.append(np.mean(train_loss))
        # predict results for test images:
        model.eval()
        test_loss = []
        for batch_idx, (image, label) in enumerate(test_loader):
            image = image.to("cuda")
            label = label.to("cuda")
            # feed forword
            output = model(image)
            # calculate loss
            loss = loss_function(output, label)
            test_loss.append(loss.item())
        print(f'Epoch [{epoch + 1}/{epoch_num}], Test Loss: {np.mean(test_loss):.4f}')
    
        epoch_test_loss.append(np.mean(test_loss))
    
    plt.plot(epoch_train_loss)
    plt.plot(epoch_test_loss)

# 6. Define Model Testing Function


In [20]:
def test_model():
    y_test = []
    
    y_test_predict = []
    
    model.eval()
    
    for image, label in test_loader:
        image = image.to("cuda")
        label = label.to("cuda")
        y = model(image).detach().cpu().numpy()[0]
        y_predict = np.argmax(y)
        y_test_predict.append(y_predict)
        y_test.append(label.cpu().numpy()[0])
    
    print(y_test_predict)
    print(y_test)
    
    cm = confusion_matrix(y_test, y_test_predict)
    print(cm)
    
    print("acc: ", accuracy_score(y_test, y_test_predict))

# 7. Execute training, and testing


In [21]:
NUM_EPOCHS = 100 # define number of epochs
train_model(NUM_EPOCHS)
test_model()

AssertionError: Torch not compiled with CUDA enabled